# Generacion de embeddings

In [1]:
# configuracion y importaciones
import pandas as pd
import numpy as np
import os
import pickle # Para guardar la lista de IDs
import sys
import re
import ast

In [2]:
# Ajuste del sys.path para importar módulos de src 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importar la función para generar embeddings
from src.generador_embeddings import generate_sbert_embeddings

c:\Users\alvar\OneDrive\Escritorio\TFM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Procesado del corpus previo generacion de embeddings
from src.procesamiento_corpus import get_text_source
df_corpus = pd.read_csv("C:\\Users\\alvar\\OneDrive\\Escritorio\\TFM\\data\\crudo\\initial_corpus.csv")
min_len_abstract = 50 # Longitud mínima de palabras para considerar un abstract sino se toma el titulo del articulo

df_corpus['processed_text'] = df_corpus.apply(
    lambda row: get_text_source(row, min_abstract_len=min_len_abstract),
    axis=1)




--- Corpus con Texto Preprocesado ('processed_text') ---
                                       paperId  \
0     4dc2617f15847af822d1f89c2e5cca39c8cdb7ad   
1     9778a564510da05080f978fcff23928ead0f1db9   
2     6a8a21cab225a428c41e3f8c38e18535f68ffacf   
3     fc88d1692a0f53f2821499fa8b8f4d049775585f   
4     9998dc44714a0721caa671243391c1ed5ecfa222   
...                                        ...   
1099  cefdd9906064c199428645512eb6834dfed72ae3   
1100  a062fbd621a9821a695b9869630feb6bfb0cf585   
1101  61b8a96cf12e961628977066d4c4807dcb526879   
1102  76a6c339e61e04069a35610faee5a523b2509f03   
1103  ccb0ca74c441aa95d127d33f736827f4ff0ab761   

                                                  title  \
0     Effect of a Machine Learning Recommender Syste...   
1     A Machine Learning Recommender System to Tailo...   
2     A Machine Learning Recommender System Based on...   
3     Matrix Factorization Collaborative-Based Recom...   
4     Smart Crop Recommender System-A Machine 

In [4]:
# limpieza del campo openAccesPdf
from src.procesamiento_corpus import extract_pdf_url_from_dict_string

df_corpus['pdf_url'] = df_corpus['openAccessPdf'].apply(extract_pdf_url_from_dict_string)
df_corpus


,paperId,title,abstract,year,authors,openAccessPdf,processed_text,pdf_url
0,4dc2617f15847af822d1f89c2e5cca39c8cdb7ad,Effect of a Machine Learning Recommender Syste...,This randomized clinical trial investigates th...,2023,"[{'authorId': '39230104', 'name': 'Jamie M. Fa...",{'url': 'https://jamanetwork.com/journals/jama...,effect of a machine learning recommender syste...,https://jamanetwork.com/journals/jamanetworkop...
1,9778a564510da05080f978fcff23928ead0f1db9,A Machine Learning Recommender System to Tailo...,Background and Objectives\nNursing homes (NHs)...,2019,"[{'authorId': '2509884', 'name': 'G. Gannod'},...",{'url': 'https://academic.oup.com/gerontologis...,background and objectives nursing homes (nhs) ...,https://academic.oup.com/gerontologist/article...
2,6a8a21cab225a428c41e3f8c38e18535f68ffacf,A Machine Learning Recommender System Based on...,Changing and moving toward online shopping has...,2020,"[{'authorId': '2172517066', 'name': 'Delshad M...",{'url': 'https://doi.org/10.22541/au.160897179...,changing and moving toward online shopping has...,https://doi.org/10.22541/au.160897179.93005705/v1
3,fc88d1692a0f53f2821499fa8b8f4d049775585f,Matrix Factorization Collaborative-Based Recom...,Saudi Arabia’s tourism sector has recently sta...,2023,"[{'authorId': '2787898', 'name': 'Reham Alabdu...",{'url': 'https://www.mdpi.com/2076-3417/13/17/...,saudi arabia’s tourism sector has recently sta...,https://www.mdpi.com/2076-3417/13/17/9574/pdf?...
4,9998dc44714a0721caa671243391c1ed5ecfa222,Smart Crop Recommender System-A Machine Learni...,Machine learning has proven its efficacy in so...,2022,"[{'authorId': '90015202', 'name': 'R. K. Ray'}...","{'url': '', 'status': None, 'license': None, '...",machine learning has proven its efficacy in so...,
...,...,...,...,...,...,...,...,...
1099,cefdd9906064c199428645512eb6834dfed72ae3,Optimization of Renewable Energy Resources in ...,This paper proposes the optimization of renewa...,2017,"[{'authorId': '144006876', 'name': 'S. S. Redd...",{'url': 'https://riverpublishers.com/journal/j...,this paper proposes the optimization of renewa...,https://riverpublishers.com/journal/journal_ar...
1100,a062fbd621a9821a695b9869630feb6bfb0cf585,PSO-Based Smart Grid Application for Sizing an...,This paper introduces an optimal sizing algori...,2016,"[{'authorId': '144816554', 'name': 'M. A. Moha...",{'url': 'https://journals.plos.org/plosone/art...,this paper introduces an optimal sizing algori...,https://journals.plos.org/plosone/article/file...
1101,61b8a96cf12e961628977066d4c4807dcb526879,Smart Integrated Renewable Energy Systems (SIR...,Technical and economic aspects of the viabilit...,2017,"[{'authorId': '30528974', 'name': 'Zeel Mahesh...",{'url': 'https://www.mdpi.com/1996-1073/10/8/1...,technical and economic aspects of the viabilit...,https://www.mdpi.com/1996-1073/10/8/1145/pdf?v...
1102,76a6c339e61e04069a35610faee5a523b2509f03,Optimization of Renewable Energy Systems: A Re...,In the contrary of decrease of fossil energy n...,2017,"[{'authorId': '112948910', 'name': 'Diriba Kaj...","{'url': '', 'status': None, 'license': None, '...",in the contrary of decrease of fossil energy n...,


In [5]:
# guardo el corpus procesado
df_corpus.to_csv("C:\\Users\\alvar\\OneDrive\\Escritorio\\TFM\\data\\Procesado\corpus_procesado.csv", index=False, encoding='utf-8')

In [6]:
# Preparar la lista de textos y la lista de IDs (en el mismo orden)
texts_to_encode = df_corpus['processed_text'].tolist()
paper_ids_ordered = df_corpus['paperId'].tolist()

print(f"Número de textos a codificar: {len(texts_to_encode)}")
print(f"Número de IDs correspondientes: {len(paper_ids_ordered)}")

Número de textos a codificar: 1104
Número de IDs correspondientes: 1104


In [ ]:
# generar los embeddings con SBERT

# definir modelo a utilizar:
# 'all-MiniLM-L6-v2' es bueno y rapido.
# Otros podrían ser: 'multi-qa-mpnet-base-dot-v1' (bueno para búsqueda semántica)
# o 'allenai-specter' (específico para papers científicos, pero más pesado)
sbert_model_name = 'all-MiniLM-L6-v2'

corpus_embeddings = generate_sbert_embeddings(
        texts_to_encode,
        model_name=sbert_model_name,
        batch_size=32 
    )

# Verificar la forma de los embeddings
# Debería ser (numero de textos, dimension del embedding)
# Para 'all-MiniLM-L6-v2', la dimensión es 384.
print(f"\nForma del array de embeddings: {corpus_embeddings.shape}")
if corpus_embeddings.shape[0] == len(texts_to_encode):
    print("El número de embeddings coincide con el número de textos.")
else:
    print("Error: El número de embeddings NO coincide con el número de textos.")

print(f"Dimensión del embedding para '{sbert_model_name}': {corpus_embeddings.shape[1]}")

Cargando el modelo SBERT: all-MiniLM-L6-v2...
Usando dispositivo: cpu
Generando embeddings para 1104 textos...


Batches: 100%|██████████| 35/35 [01:11<00:00,  2.05s/it]

Embeddings generados con forma: (1104, 384)

Forma del array de embeddings: (1104, 384)
El número de embeddings coincide con el número de textos.
Dimensión del embedding para 'all-MiniLM-L6-v2': 384


In [ ]:
# guardar embeddings y los ids
# IMP!! guardar los embeddings en el mismo orden que los ids para despues poder buscarlos correctamente

embeddings_file_path = os.path.join("C:\\Users\\alvar\\OneDrive\\Escritorio\\TFM\\data\\Embeddings", f'corpus_embeddings_{sbert_model_name.replace("/", "_")}.npy')
ids_file_path = os.path.join("C:\\Users\\alvar\\OneDrive\\Escritorio\\TFM\\data\\Embeddings", f'corpus_paper_ids_{sbert_model_name.replace("/", "_")}.pkl')

# Guardar embeddings (array NumPy)
np.save(embeddings_file_path, corpus_embeddings)
print(f"\nEmbeddings guardados en: {embeddings_file_path}")

# Guardar lista de paperIds 
with open(ids_file_path, 'wb') as f:
    pickle.dump(paper_ids_ordered, f)
print(f"Lista de Paper IDs ordenada guardada en: {ids_file_path}")


Embeddings guardados en: C:\Users\alvar\OneDrive\Escritorio\TFM\data\Embeddings\corpus_embeddings_all-MiniLM-L6-v2.npy
Lista de Paper IDs ordenada guardada en: C:\Users\alvar\OneDrive\Escritorio\TFM\data\Embeddings\corpus_paper_ids_all-MiniLM-L6-v2.pkl
